<h1>Basics of Mobile Robotics project</h1>


In [1]:
USE_THYMIO = False

import tdmclient.notebook
if USE_THYMIO:
    await tdmclient.notebook.start()
from global_navigation import *

print("Hello")


Hello


In [ ]:
@tdmclient.notebook.sync_var
def motor_go(left,right):
    global motor_left_target, motor_right_target
    motor_left_target = [left]
    motor_right_target = [right]
    
@tdmclient.notebook.sync_var
def motor_stop():
    global motor_left_target,motor_right_target
    motor_left_target = 0
    motor_right_target = 0
    
@tdmclient.notebook.sync_var
def print_prox_value():
    return prox_horizontal[]

In [3]:
state = 1
'''
     States of the global FSM (finite state machine)
     
     1 -> computer vision and global path planning
     2 -> path following
     3 -> obstacle avoidance
     4 -> target reached
'''
print("eddai")

while True:
    if state == 1:
        print("MAIN FSM -> computer vision")
        # ToDo
        
        nodes = [(1,2),(3,4),(5,6),(10,11)]

        global_nav = Global_Navigation(nodes)

        global_nav.add_edge((1,2),(3,4))
        global_nav.add_edge((1,2),(5,6))
        global_nav.add_edge((3,4),(10,11))
        global_nav.add_edge((10,11),(5,6))


        start_node = (1,2)
        target_node= (10,11)

        
        # next state -> path planning
        state = 2
        
    elif state == 2:
        print("MAIN FSM -> path planning")
        path = global_nav.path_planning(start_node,target_node)
        
        # next state -> path following
        state = 3
    elif state == 3:
        
        
    elif state == 4:
        print("MAIN FSM -> terminated")
        break


eddai
MAIN FSM -> computer vision
MAIN FSM -> path planning
We found the following best path with a value of 12.727922061357855.
(1, 2) -> (3, 4) -> (10, 11)
MAIN FSM -> terminated
